__Тепловая карта активности пользователя в ПО:__

In [75]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, date, time

In [76]:
# путь к двум выгрузкам из БД
file_path_activities = r'C:\Users\kakorin_is\Desktop\JupiterNotebook_files\Активности всех дата начала с 20.11.2022 по сейчас (дату создания и завершения чекать, у с_з дату начала).xlsx'
file_path_to_user_id = r'C:\Users\kakorin_is\Desktop\Свод\юристы РОО.xlsx'

year_for_checking = 2023     # указать год
month_start_for_checking = 1 # указать номер месяца с которого начинамем включительно
month_end_for_checking = 3   # указать номер месяца до которого деламе выборку, не включая его

# в условиях реальной работы False
demonstration_mode = True

In [77]:
activities = pd.read_excel(file_path_activities, index_col=None)
activities.columns

Index(['Мероприятие', 'Индикатор', 'Тип', 'Тип активности Outbound', 'Статус',
       'Начало', 'Фактическое окончание', 'Результат', 'Соотношение к ПЗ',
       'Завершена с просрочкой', 'Клиент (ФЛ)', 'КМБ', 'Комментарии',
       '№ договора', 'Судебное дело', 'Адрес',
       'Дней просрочки на момент создания', 'Ответственный по клиенту',
       'Центр сбора', 'Номер дела о банкротстве', 'Ответственный',
       'Дата создания', 'Дата передачи', 'Общий комментарий', 'Описание',
       'Иная судебная работа', 'Дата публичных торгов (принятие на баланс)'],
      dtype='object')

In [78]:
activities.shape

(6037, 27)

In [79]:
# Обезличиваю инф. по Ответственным, т.к. режим дменострации. У каждого ФИО свой юзер_ид

if demonstration_mode == True:
    user_id_data = pd.read_excel(file_path_to_user_id, index_col=None)
    print(user_id_data.columns) # печатаю чтобы было понимание что содержится в файле
    
    names_list = user_id_data['Ответственный'].values
    user_id_list = user_id_data['id юриста'].values 
    dict_id = dict(zip(names_list, user_id_list))
    activities['Ответственный'] = activities['Ответственный'].apply(lambda x: dict_id[x])

activities['Ответственный'].value_counts()

Index(['Ответственный', 'роо', 'id юриста'], dtype='object')


100000023    963
100000019    828
100000012    803
100000021    737
100000015    693
100000016    680
100000011    659
100000014    601
100000005     49
100000020     24
Name: Ответственный, dtype: int64

In [80]:
# из всех колонок для нас важны только эти
usefull_cols = ['Мероприятие','Ответственный','Тип', 'Дата создания']

activities = activities.loc[:, usefull_cols]

activities.head(4)

,Мероприятие,Ответственный,Тип,Дата создания
0,1-1JR58QN,100000011,Судебное заседание,2023-02-28 10:33:39
1,1-1J6OV3Q,100000011,Судебное заседание,2023-02-15 16:47:20
2,1-1IIMMX8,100000011,Судебное заседание,2023-02-02 16:58:57
3,1-1JQN0C7,100000011,Судебное заседание,2023-02-27 15:14:51


In [81]:
# смотрим типы данных в колонках
activities.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6037 entries, 0 to 6036
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Мероприятие    6037 non-null   object        
 1   Ответственный  6037 non-null   int64         
 2   Тип            6037 non-null   object        
 3   Дата создания  6037 non-null   datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 188.8+ KB


In [82]:
activities['Дата создания испр'] = activities['Дата создания'].apply(lambda x: x.date())

In [83]:
activities.head(4)

,Мероприятие,Ответственный,Тип,Дата создания,Дата создания испр
0,1-1JR58QN,100000011,Судебное заседание,2023-02-28 10:33:39,2023-02-28
1,1-1J6OV3Q,100000011,Судебное заседание,2023-02-15 16:47:20,2023-02-15
2,1-1IIMMX8,100000011,Судебное заседание,2023-02-02 16:58:57,2023-02-02
3,1-1JQN0C7,100000011,Судебное заседание,2023-02-27 15:14:51,2023-02-27


In [84]:
# создаем отдельную колонку со временем суток, взятую из созданной активности, чтобы в дальнейшем разделить на категории

activities['Время'] = activities['Дата создания'].apply(lambda x: x.time())

In [85]:
# выделим отдельно год и месяц

activities['Год'] = activities['Дата создания'].apply(lambda x: x.year)
activities['Месяц'] = activities['Дата создания'].apply(lambda x: x.month)
activities.head(3)

,Мероприятие,Ответственный,Тип,Дата создания,Дата создания испр,Время,Год,Месяц
0,1-1JR58QN,100000011,Судебное заседание,2023-02-28 10:33:39,2023-02-28,10:33:39,2023,2
1,1-1J6OV3Q,100000011,Судебное заседание,2023-02-15 16:47:20,2023-02-15,16:47:20,2023,2
2,1-1IIMMX8,100000011,Судебное заседание,2023-02-02 16:58:57,2023-02-02,16:58:57,2023,2


In [86]:
activities.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6037 entries, 0 to 6036
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Мероприятие         6037 non-null   object        
 1   Ответственный       6037 non-null   int64         
 2   Тип                 6037 non-null   object        
 3   Дата создания       6037 non-null   datetime64[ns]
 4   Дата создания испр  6037 non-null   object        
 5   Время               6037 non-null   object        
 6   Год                 6037 non-null   int64         
 7   Месяц               6037 non-null   int64         
dtypes: datetime64[ns](1), int64(3), object(4)
memory usage: 377.4+ KB


In [87]:
# проверяю возможность сравнения времени - все работает
to_search = time(hour=22, minute=0, second=0, microsecond=0, tzinfo=None)
activities.loc[activities['Время'] > to_search]

,Мероприятие,Ответственный,Тип,Дата создания,Дата создания испр,Время,Год,Месяц
46,1-1JXJH4J,100000012,Судебное заседание,2023-03-01 23:30:39,2023-03-01,23:30:39,2023,3
101,1-1J12BYN,100000012,Судебное заседание,2023-02-12 23:49:12,2023-02-12,23:49:12,2023,2
102,1-1GVZ82H,100000012,Судебное заседание,2023-01-08 23:55:46,2023-01-08,23:55:46,2023,1
104,1-1J11UJ2,100000012,Судебное заседание,2023-02-12 23:26:06,2023-02-12,23:26:06,2023,2
107,1-1J11UHN,100000012,Судебное заседание,2023-02-12 23:10:44,2023-02-12,23:10:44,2023,2
...,...,...,...,...,...,...,...,...
5923,1-1F0L92X,100000012,Запрос на оплату госпошлины,2022-11-28 23:46:54,2022-11-28,23:46:54,2022,11
5927,1-1F0L92R,100000012,Запрос расчета задолженности,2022-11-28 23:45:49,2022-11-28,23:45:49,2022,11
5928,1-1F0L92N,100000012,Запрос копий документов,2022-11-28 23:44:50,2022-11-28,23:44:50,2022,11
5966,1-1DPQ28A,100000015,Судебное заседание,2022-11-03 23:00:55,2022-11-03,23:00:55,2022,11


In [88]:
# функция преобразования времени в категории, будем понимать кто работает сверх рабочего времени и выходные

def find_time_periods(some_time):
    time_24_00 = time(hour=0, minute=0, second=0, microsecond=0, tzinfo=None)
    time_23_59 = time(hour=23, minute=59, second=59, microsecond=0, tzinfo=None)
    time_22_00 = time(hour=22, minute=0, second=0, microsecond=0, tzinfo=None)
    time_20_00 = time(hour=20, minute=0, second=0, microsecond=0, tzinfo=None)
    time_18_30 = time(hour=18, minute=30, second=0, microsecond=0, tzinfo=None)
    time_18_00 = time(hour=18, minute=0, second=0, microsecond=0, tzinfo=None)
    time_06_00 = time(hour=6, minute=0, second=0, microsecond=0, tzinfo=None)
    time_07_30 = time(hour=7, minute=30, second=0, microsecond=0, tzinfo=None)
    time_8_50 = time(hour=8, minute=50, second=0, microsecond=0, tzinfo=None)
    
    if (some_time > time_24_00) and (some_time <= time_06_00):
        return '00:00 - 06:00'
    if (some_time > time_06_00) and (some_time <= time_07_30):
        return '06:00 - 07:30'
    if (some_time > time_07_30) and (some_time <= time_8_50):
        return '07:30 - 08:50'
    if (some_time > time_8_50) and (some_time <= time_18_00):
        return '08:50 - 18:00'
    if (some_time > time_18_00) and (some_time <= time_18_30):
        return '18:00 - 18:30'
    if (some_time > time_18_30) and (some_time <= time_20_00):
        return '18:30 - 20:00'
    if (some_time > time_20_00) and (some_time <= time_22_00):
        return '20:00 - 22:30'
    if (some_time > time_22_00) and (some_time <= time_23_59):
        return '22:00 - 23:59'
    
    # вернем ошибку, если вдруг выпало из условий
    return 'error'


activities['Статистический период суток'] = activities['Время'].apply(lambda x: find_time_periods(x))
activities.head(5)

,Мероприятие,Ответственный,Тип,Дата создания,Дата создания испр,Время,Год,Месяц,Статистический период суток
0,1-1JR58QN,100000011,Судебное заседание,2023-02-28 10:33:39,2023-02-28,10:33:39,2023,2,08:50 - 18:00
1,1-1J6OV3Q,100000011,Судебное заседание,2023-02-15 16:47:20,2023-02-15,16:47:20,2023,2,08:50 - 18:00
2,1-1IIMMX8,100000011,Судебное заседание,2023-02-02 16:58:57,2023-02-02,16:58:57,2023,2,08:50 - 18:00
3,1-1JQN0C7,100000011,Судебное заседание,2023-02-27 15:14:51,2023-02-27,15:14:51,2023,2,08:50 - 18:00
4,1-1IIZ21O,100000011,Судебное заседание,2023-02-02 18:07:17,2023-02-02,18:07:17,2023,2,18:00 - 18:30


In [89]:
activities['Статистический период суток'].value_counts()

08:50 - 18:00    5054
22:00 - 23:59     222
00:00 - 06:00     211
18:00 - 18:30     210
18:30 - 20:00     139
20:00 - 22:30     128
07:30 - 08:50      70
06:00 - 07:30       3
Name: Статистический период суток, dtype: int64

In [90]:
# должно быть пусто при корректном описании условий в функции

activities.loc[activities['Статистический период суток'] == 'error']

,Мероприятие,Ответственный,Тип,Дата создания,Дата создания испр,Время,Год,Месяц,Статистический период суток


__Все прошло успешно.__

Делаем выборку из общих данных с учетом интересующего периода (месяцы и год).

In [91]:
activities_cuted = activities.loc[(activities['Год'] == year_for_checking) & \
                                  (activities['Месяц'] >= month_start_for_checking) & \
                                  (activities['Месяц'] < month_end_for_checking)]

activities_cuted.shape

(3427, 9)

In [92]:
# проверял все ли корректно
activities_cuted['Год'].value_counts()

2023    3427
Name: Год, dtype: int64

In [93]:
# проверял все ли корректно
activities_cuted['Месяц'].value_counts()

2    1881
1    1546
Name: Месяц, dtype: int64

__После того, как был создан столбец с категориями времени суток, перекодирую этот столбец с помощью one hot encoding:__

In [94]:
activities_cuted_dummies = pd.get_dummies(activities_cuted['Статистический период суток'])
activities_cuted = pd.merge(activities_cuted, activities_cuted_dummies, left_index=True, right_index=True, how='outer')
activities_cuted.head(5)

,Мероприятие,Ответственный,Тип,Дата создания,Дата создания испр,Время,Год,Месяц,Статистический период суток,00:00 - 06:00,06:00 - 07:30,07:30 - 08:50,08:50 - 18:00,18:00 - 18:30,18:30 - 20:00,20:00 - 22:30,22:00 - 23:59
0,1-1JR58QN,100000011,Судебное заседание,2023-02-28 10:33:39,2023-02-28,10:33:39,2023,2,08:50 - 18:00,0,0,0,1,0,0,0,0
1,1-1J6OV3Q,100000011,Судебное заседание,2023-02-15 16:47:20,2023-02-15,16:47:20,2023,2,08:50 - 18:00,0,0,0,1,0,0,0,0
2,1-1IIMMX8,100000011,Судебное заседание,2023-02-02 16:58:57,2023-02-02,16:58:57,2023,2,08:50 - 18:00,0,0,0,1,0,0,0,0
3,1-1JQN0C7,100000011,Судебное заседание,2023-02-27 15:14:51,2023-02-27,15:14:51,2023,2,08:50 - 18:00,0,0,0,1,0,0,0,0
4,1-1IIZ21O,100000011,Судебное заседание,2023-02-02 18:07:17,2023-02-02,18:07:17,2023,2,18:00 - 18:30,0,0,0,0,1,0,0,0


In [95]:
# Также хочу отдельно видеть день недели, чтобы окрасить работу в выходные

activities_cuted['День недели'] = activities_cuted['Дата создания испр'].apply(lambda x: x.strftime('%a'))
activities_cuted.head(5)

,Мероприятие,Ответственный,Тип,Дата создания,Дата создания испр,Время,Год,Месяц,Статистический период суток,00:00 - 06:00,06:00 - 07:30,07:30 - 08:50,08:50 - 18:00,18:00 - 18:30,18:30 - 20:00,20:00 - 22:30,22:00 - 23:59,День недели
0,1-1JR58QN,100000011,Судебное заседание,2023-02-28 10:33:39,2023-02-28,10:33:39,2023,2,08:50 - 18:00,0,0,0,1,0,0,0,0,Tue
1,1-1J6OV3Q,100000011,Судебное заседание,2023-02-15 16:47:20,2023-02-15,16:47:20,2023,2,08:50 - 18:00,0,0,0,1,0,0,0,0,Wed
2,1-1IIMMX8,100000011,Судебное заседание,2023-02-02 16:58:57,2023-02-02,16:58:57,2023,2,08:50 - 18:00,0,0,0,1,0,0,0,0,Thu
3,1-1JQN0C7,100000011,Судебное заседание,2023-02-27 15:14:51,2023-02-27,15:14:51,2023,2,08:50 - 18:00,0,0,0,1,0,0,0,0,Mon
4,1-1IIZ21O,100000011,Судебное заседание,2023-02-02 18:07:17,2023-02-02,18:07:17,2023,2,18:00 - 18:30,0,0,0,0,1,0,0,0,Thu


In [96]:
activities_cuted['День недели'].value_counts()

Wed    771
Mon    715
Tue    664
Thu    543
Fri    517
Sun    197
Sat     20
Name: День недели, dtype: int64

In [97]:
# Monday – понедельник;
# Tuesday  – вторник;
# Wednesday  – среда;
# Thursday  – четверг;
# Friday  – пятница;
# Saturday – суббота;
# Sunday – воскресенье.

In [98]:
# по умолчанию ноль: 0 значит будний день, 1 - выходной
activities_cuted['Работы в выходные'] = 0

# если день недели суббота или вскр, то ноль меняем на 1 (0 и 1 использую чтобы потом сразу было удобно считать)
activities_cuted['Работы в выходные'] = np.where((activities_cuted['День недели'] == 'Sun'), 1, activities_cuted['Работы в выходные'])                                      
activities_cuted['Работы в выходные'] = np.where((activities_cuted['День недели'] == 'Sat'), 1, activities_cuted['Работы в выходные'])

activities_cuted.groupby(['Работы в выходные','День недели']).agg({'Дата создания испр': 'count'})

Дата создания испр
Работы в выходные День недели                    
0                 Fri                         517
                  Mon                         715
                  Thu                         543
                  Tue                         664
                  Wed                         771
1                 Sat                          20
                  Sun                         197

В полученной таблице содержатся все возможные активности (строки), которые заполнялись Ответственными, в т.ч. встречается одна и та же дата, но с разным типом активности.<br>

Меня же интересуют активные среди пользователей дни, поэтому буду группировать по дням.

In [99]:
pre_result = activities_cuted.groupby(['Ответственный','Дата создания испр']).agg({'00:00 - 06:00': 'sum', '06:00 - 07:30': 'sum',\
                                                                      '07:30 - 08:50': 'sum', '08:50 - 18:00': 'sum',\
                                                                      '18:00 - 18:30': 'sum', '18:30 - 20:00': 'sum',\
                                                                      '20:00 - 22:30': 'sum', '22:00 - 23:59': 'sum',\
                                                                      'Работы в выходные': 'sum'})

In [100]:
pre_result

00:00 - 06:00  06:00 - 07:30  07:30 - 08:50  \
Ответственный Дата создания испр                                                
100000005     2023-02-09                      0              0              0   
              2023-02-10                      0              0              0   
              2023-02-15                      0              0              0   
100000011     2023-01-09                      0              0              0   
              2023-01-10                      0              0              0   
...                                         ...            ...            ...   
100000023     2023-02-20                      0              0              0   
              2023-02-21                      0              0              0   
              2023-02-22                      0              0              0   
              2023-02-27                      0              0              0   
              2023-02-28                      0              0              0   

                                  08:50 - 18:00  18:00 - 18:30  18:30 - 20:00  \
Ответственный Дата создания испр                                                
100000005     2023-02-09                     12              0              0   
              2023-02-10                      2              0              0   
              2023-02-15                      4              8              0   
100000011     2023-01-09                     10              0              0   
              2023-01-10                      6              0              0   
...                                         ...            ...            ...   
100000023     2023-02-20                     12              0              0   
              2023-02-21                     14              0              0   
              2023-02-22                     13              0              0   
              2023-02-27                     11              2              2   
              2023-02-28                      4              1              0   

                                  20:00 - 22:30  22:00 - 23:59  \
Ответственный Дата создания испр                                 
100000005     2023-02-09                      0              0   
              2023-02-10                      0              0   
              2023-02-15                      0              0   
100000011     2023-01-09                      0              0   
              2023-01-10                      0              0   
...                                         ...            ...   
100000023     2023-02-20                      0              0   
              2023-02-21                      0              0   
              2023-02-22                      0              0   
              2023-02-27                      0              0   
              2023-02-28                      0              0   

                                  Работы в выходные  
Ответственный Дата создания испр                     
100000005     2023-02-09                          0  
              2023-02-10                          0  
              2023-02-15                          0  
100000011     2023-01-09                          0  
              2023-01-10                          0  
...                                             ...  
100000023     2023-02-20                          0  
              2023-02-21                          0  
              2023-02-22                          0  
              2023-02-27                          0  
              2023-02-28                          0  

[275 rows x 9 columns]

In [101]:
# после группировки вывожу Дату и Ответственного из индексов в колонку
pre_result.reset_index(drop=False, inplace= True )

In [102]:
pre_result.shape

(275, 11)

In [103]:
pre_result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 275 entries, 0 to 274
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Ответственный       275 non-null    int64 
 1   Дата создания испр  275 non-null    object
 2   00:00 - 06:00       275 non-null    uint8 
 3   06:00 - 07:30       275 non-null    uint8 
 4   07:30 - 08:50       275 non-null    uint8 
 5   08:50 - 18:00       275 non-null    uint8 
 6   18:00 - 18:30       275 non-null    uint8 
 7   18:30 - 20:00       275 non-null    uint8 
 8   20:00 - 22:30       275 non-null    uint8 
 9   22:00 - 23:59       275 non-null    uint8 
 10  Работы в выходные   275 non-null    int64 
dtypes: int64(2), object(1), uint8(8)
memory usage: 8.7+ KB


In [104]:
pre_result['Месяц'] = pre_result['Дата создания испр'].apply(lambda x: x.month)
pre_result.head(2)

,Ответственный,Дата создания испр,00:00 - 06:00,06:00 - 07:30,07:30 - 08:50,08:50 - 18:00,18:00 - 18:30,18:30 - 20:00,20:00 - 22:30,22:00 - 23:59,Работы в выходные,Месяц
0,100000005,2023-02-09,0,0,0,12,0,0,0,0,0,2
1,100000005,2023-02-10,0,0,0,2,0,0,0,0,0,2


__Значения в колонках до этого считались как сумма всех активностей в сгруппированном дне, меня же интересует кол-во дней, когда работник был активен в указанных категориях времени суток.__

Для этого все числа больше нуля перепишу в единицы (аналог флага бинарных данных), нули оставлю нулями - активности в этих категориях не было.

In [105]:
# список колонок, в которых будем заменять числа
cols = pre_result.columns[2:-1]
cols

Index(['00:00 - 06:00', '06:00 - 07:30', '07:30 - 08:50', '08:50 - 18:00',
       '18:00 - 18:30', '18:30 - 20:00', '20:00 - 22:30', '22:00 - 23:59',
       'Работы в выходные'],
      dtype='object')

In [106]:
# замению числа в выбранных колонках
for i in cols:
    pre_result[i] = np.where((pre_result[i] > 0), 1, pre_result[i])

pre_result

,Ответственный,Дата создания испр,00:00 - 06:00,06:00 - 07:30,07:30 - 08:50,08:50 - 18:00,18:00 - 18:30,18:30 - 20:00,20:00 - 22:30,22:00 - 23:59,Работы в выходные,Месяц
0,100000005,2023-02-09,0,0,0,1,0,0,0,0,0,2
1,100000005,2023-02-10,0,0,0,1,0,0,0,0,0,2
2,100000005,2023-02-15,0,0,0,1,1,0,0,0,0,2
3,100000011,2023-01-09,0,0,0,1,0,0,0,0,0,1
4,100000011,2023-01-10,0,0,0,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
270,100000023,2023-02-20,0,0,0,1,0,0,0,0,0,2
271,100000023,2023-02-21,0,0,0,1,0,0,0,0,0,2
272,100000023,2023-02-22,0,0,0,1,0,0,0,0,0,2
273,100000023,2023-02-27,0,0,0,1,1,1,0,0,0,2


In [107]:
pre_result['Работы в выходные'].value_counts()

0    263
1     12
Name: Работы в выходные, dtype: int64

In [108]:
# Для удобства восприятия инф. в таблице перепишу номера месяцев в строковые значения:

month_dict = {1: 'Январь',
              2: 'Февраль',
              3: 'Март',
              4: 'Апрель',
              5: 'Май',
              6: 'Июнь',
              7: 'Июль',
              8: 'Август',
              9: 'Сентябрь',
              10: 'Октябрь',
              11: 'Ноябрь',
              12: 'Декабрь'
    
}

pre_result['Месяц'] = pre_result['Месяц'].apply(lambda x: month_dict[x])

In [109]:
final_result = pre_result.groupby(['Ответственный','Месяц']).agg({'00:00 - 06:00': 'sum', '06:00 - 07:30': 'sum',\
                                                                      '07:30 - 08:50': 'sum', '08:50 - 18:00': 'sum',\
                                                                      '18:00 - 18:30': 'sum', '18:30 - 20:00': 'sum',\
                                                                      '20:00 - 22:30': 'sum', '22:00 - 23:59': 'sum',\
                                                                      'Работы в выходные': 'sum'})
final_result

00:00 - 06:00  06:00 - 07:30  07:30 - 08:50  \
Ответственный Месяц                                                  
100000005     Февраль              0              0              0   
100000011     Февраль              0              0              0   
              Январь               0              0              0   
100000012     Февраль              7              0              0   
              Январь               4              0              0   
100000014     Февраль              0              0              1   
              Январь               0              0              3   
100000015     Февраль              3              0              3   
              Январь               0              0              0   
100000016     Февраль              0              0              0   
              Январь               0              0              4   
100000019     Февраль              1              0              4   
              Январь               0              1              3   
100000021     Февраль              0              0              1   
              Январь               0              0              0   
100000023     Февраль              0              0              0   
              Январь               0              0              0   

                       08:50 - 18:00  18:00 - 18:30  18:30 - 20:00  \
Ответственный Месяц                                                  
100000005     Февраль              3              1              0   
100000011     Февраль             18              6              3   
              Январь              17              0              0   
100000012     Февраль             12              1              0   
              Январь              12              1              1   
100000014     Февраль             17              1              0   
              Январь              17              0              0   
100000015     Февраль             17              4              5   
              Январь              13              3              3   
100000016     Февраль             15              2              0   
              Январь              16              2              0   
100000019     Февраль             20              3              2   
              Январь              18              0              2   
100000021     Февраль             16              3              0   
              Январь              16              1              0   
100000023     Февраль             18              4              3   
              Январь              18              0              0   

                       20:00 - 22:30  22:00 - 23:59  Работы в выходные  
Ответственный Месяц                                                     
100000005     Февраль              0              0                  0  
100000011     Февраль              0              0                  0  
              Январь               0              0                  0  
100000012     Февраль              1              6                  1  
              Январь               2              3                  3  
100000014     Февраль              0              0                  0  
              Январь               0              0                  0  
100000015     Февраль              2              8                  1  
              Январь               0              1                  0  
100000016     Февраль              2              1                  0  
              Январь               0              0                  0  
100000019     Февраль              5              3                  2  
              Январь               4              6                  4  
100000021     Февраль              0              0                  0  
              Январь               0              0                  0  
100000023     Февраль              2              0                  0  
              Январь               1           

In [112]:
# сохраняем в эксель
writer = pd.ExcelWriter(f'тепловая карта активности пользователей в ПО в период с {month_dict[month_start_for_checking]} по \
{month_dict[month_end_for_checking]} {year_for_checking}г.xlsx') 

final_result.to_excel(writer) 
writer.save()